<a href="https://colab.research.google.com/github/FernandoAltamirano/sistemas-inteligentes/blob/main/Dropout_en_la_capa_visible.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.constraints import maxnorm
from tensorflow.keras.optimizers import SGD
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Dropout en la capa visible

In [6]:
dataframe = read_csv("cd.csv",header=0)

In [7]:
dataframe.head()

,Unnamed: 0,WTT,PTI,EQW,SBI,LQE,QWG,FDJ,PJF,HQE,NXJ,TARGET CLASS
0,0,0.913917,1.162073,0.567946,0.755464,0.780862,0.352608,0.759697,0.643798,0.879422,1.231409,1
1,1,0.635632,1.003722,0.535342,0.825645,0.924109,0.648450,0.675334,1.013546,0.621552,1.492702,0
2,2,0.721360,1.201493,0.921990,0.855595,1.526629,0.720781,1.626351,1.154483,0.957877,1.285597,0
3,3,1.234204,1.386726,0.653046,0.825624,1.142504,0.875128,1.409708,1.380003,1.522692,1.153093,1
4,4,1.279491,0.949750,0.627280,0.668976,1.232537,0.703727,1.115596,0.646691,1.463812,1.419167,1


In [8]:
dataframe.pop("Unnamed: 0")

0        0
1        1
2        2
3        3
4        4
      ... 
995    995
996    996
997    997
998    998
999    999
Name: Unnamed: 0, Length: 1000, dtype: int64

In [9]:
dataframe.head()

,WTT,PTI,EQW,SBI,LQE,QWG,FDJ,PJF,HQE,NXJ,TARGET CLASS
0,0.913917,1.162073,0.567946,0.755464,0.780862,0.352608,0.759697,0.643798,0.879422,1.231409,1
1,0.635632,1.003722,0.535342,0.825645,0.924109,0.648450,0.675334,1.013546,0.621552,1.492702,0
2,0.721360,1.201493,0.921990,0.855595,1.526629,0.720781,1.626351,1.154483,0.957877,1.285597,0
3,1.234204,1.386726,0.653046,0.825624,1.142504,0.875128,1.409708,1.380003,1.522692,1.153093,1
4,1.279491,0.949750,0.627280,0.668976,1.232537,0.703727,1.115596,0.646691,1.463812,1.419167,1


In [10]:
datos = dataframe.values

In [11]:
X = datos[:,0:10].astype(float)

In [12]:
X

array([[0.91391733, 1.16207271, 0.56794585, ..., 0.64379756, 0.87942209,
        1.23140944],
       [0.6356319 , 1.00372163, 0.53534235, ..., 1.01354599, 0.62155222,
        1.4927016 ],
       [0.72135981, 1.20149262, 0.92198973, ..., 1.15448315, 0.95787702,
        1.28559679],
       ...,
       [1.13546983, 0.98246233, 0.78190516, ..., 0.38958442, 0.91919143,
        1.385504  ],
       [1.08489449, 0.86176917, 0.40715771, ..., 1.06133794, 1.27745578,
        1.18806277],
       [0.83745954, 0.96118352, 0.41700599, ..., 0.90796187, 1.25718998,
        1.36483726]])

In [13]:
Y = datos[:,10]

In [14]:
Y

array([1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 1., 0., 0., 1., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 1., 1., 1., 1., 1.,
       1., 0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0.,
       1., 1., 1., 1., 0., 0., 1., 1., 1., 0., 0., 0., 0., 1., 1., 1., 0.,
       1., 0., 0., 0., 1., 0., 1., 0., 0., 1., 1., 1., 0., 1., 1., 1., 0.,
       0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0.,
       1., 0., 1., 1., 0., 0., 1., 0., 0., 1., 1., 0., 1., 1., 0., 1., 1.,
       0., 1., 1., 1., 1., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 1., 1.,
       1., 1., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 1., 0., 1., 0., 1.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 1., 1.,
       1., 1., 0., 0., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 0., 0.,
       0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 1., 1., 0.,
       1., 0., 0., 0., 0., 1., 0., 1., 1., 0., 1., 1., 0., 0., 0., 0., 0.,
       1., 0., 1., 0., 0.

In [15]:
codificador = LabelEncoder()

In [16]:
codificador.fit(Y)

LabelEncoder()

In [17]:
Y_codificado = codificador.transform(Y)

## Modelo Base

In [18]:
def crear_modelo():
  modelo = Sequential()
  modelo.add(Dropout(0.2,input_shape=(10,)))
  modelo.add(Dense(60, activation='relu',kernel_constraint=maxnorm(3)))
  modelo.add(Dense(30, activation='relu',kernel_constraint=maxnorm(3)))
  modelo.add(Dense(1, activation='sigmoid'))
  sgd = SGD(learning_rate=0.1,momentum=0.9)
  modelo.compile(loss='binary_crossentropy',optimizer=sgd,metrics=['accuracy'])
  return modelo

In [19]:
estimadores = []
estimadores.append(('estandarizacion',StandardScaler()))
estimadores.append(('mlp',KerasClassifier(build_fn=crear_modelo,epochs=350,
batch_size=16,verbose=0)))
canal = Pipeline(estimadores)
kfold = StratifiedKFold(n_splits=10,shuffle=True)
resultados = cross_val_score(canal, X, Y_codificado, cv=kfold)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  after removing the cwd from sys.path.


In [20]:
resultados

array([0.94      , 0.95999998, 0.85000002, 0.94      , 0.88999999,
       0.98000002, 0.95999998, 0.88      , 0.88999999, 0.94999999])

In [21]:
print("Modelo Con Dropout en la Entrada: %.2f%% (%.2f%%)"%
(resultados.mean()*100,resultados.std()*100))

Modelo Con Dropout en la Entrada: 92.40% (4.08%)
